In [3]:
from flask import Flask
from flask import request
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State

# server = Flask(__name__)
# app = dash.Dash(
#     __name__,
#     server = server,
#     serve_locally = False,
#     requests_pathname_prefix = "/plotary/dash/",
# )

app = dash.Dash(__name__,url_base_pathname="/plotary/dash/")

app.layout = html.Div([
    dcc.Location(id='url', refresh=False),
        html.Div([
#         html.A([
#             html.Div(['goto man-es page'],id="logo")
#         ],href='https://man-es.com'),
        html.Div([
            html.H1('Keine Auswertung ausgewählt. ')
        ],id="description"),
        # This Link does not seem to be causing an invocation of the callback 
        dcc.Link('Navigate to https://man-es.com', href='https://man-es.com'),
        html.Br(),
        # These 2 Links DO seem to be causing an invocation of the callback 
        dcc.Link('Navigate to "/plotary/dash/"', href='/plotary/dash/'),
        html.Br(),
        dcc.Link('Navigate to "/plotary/"', href='/plotary/'),
            
    ],
    id="navbar")]
)

@app.callback(Output('description', 'children'),
              [Input('url', 'pathname')])
def display_page(pathname):
    print(f'entering callback for pathname {pathname}')
    return html.Div([
        html.H1('Auswertung Nr {}'.format(pathname))
    ])

# if __name__ == '__main__':
#     app.run_server(port=8882,debug=False)

In [32]:
import numpy as np
import pandas as pd
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
from dash.exceptions import PreventUpdate

import dash_table

def make_dt(table_id,df,display_headers=True):
    dt = dash_table.DataTable(
        id=table_id,
        columns=[{"name": i, "id": i} for i in df.columns],
        data=df.to_dict('records'),
        filter_action="custom",
        sort_action="native",
        sort_mode="multi",
        column_selectable="single",
        row_selectable="multi",
#         style_cell={
#             'width': '{}%'.format(len(df.columns)),
#             'textOverflow': 'ellipsis',
#             'overflow': 'hidden'
#         }
    )
    if not display_headers:
        dt.style_header = {'display': 'none'}        
    return dt

    
def make_text_centered_div(text):    
    col_inner_style = {
        'margin':'auto',
        'word-break':'break-all',
        'word-wrap': 'break-word',
        'text-align':'center'
    }
    return html.Div([text],style=col_inner_style)

# make simple DataFrame
x = np.arange(0,20)
y = x**2
df = pd.DataFrame({'x':x,'y':y})

# create an "aggregates" DataFrame
df_agg = pd.DataFrame({'x':[int(df.x.mean())],'y':[df.y.sum()]})

# create dash_tables
dt = make_dt('mydt',df)
dt_agg = make_dt('mydt_agg',df_agg,display_headers=False)
agg_title = make_text_centered_div("aggregates")

# make app
app = dash.Dash(__name__)
app.layout = html.Div([dt,agg_title,dt_agg])
app.title = "My DataFrame With Aggs"

@app.callback(
    Output('mydt_agg','data'),
    [Input('mydt', 'rows'),
     Input('mydt', 'selected_row_indices')])
def selected_callback(rows,selected_row_indices):
    try:
        df_selected_rows=pd.DataFrame(rows)
        df_new_agg = pd.DataFrame({'x':[df_selected_rows.x.mean()],'y':[df_selected_rows.y.sum()]})
        d = df_new_agg.to_dict('records')
        return d
    except Exception as e:
        print(str(e))
        print(rows,selected_row_indices)
        raise PreventUpdate()

@app.callback(
    Output('mydt','data'),
#     [Input('mydt','filter_query')]
    [Input('mydt','derived_filter_query_structure')]
)
def process_filter(filter_query):
    print(filter_query,type(filter_query))
    return df.to_dict('records')
    if filter_query is None or len(filter_query)<=0:
        return df.to_dict('records')
    df_temp = df.copy()
    #{x} > 10 && {y} > 50
    fq = str(filter_query).replace('"','').replace('{',"").replace('}',"").replace('&&','and')
    print(fq)
    df_temp = df_temp.query(fq)
    return df_temp.to_dict('records')


app.run_server(host='127.0.0.1',port=8884)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8884/ (Press CTRL+C to quit)
127.0.0.1 - - [15/Jul/2020 16:20:44] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Jul/2020 16:20:44] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [15/Jul/2020 16:20:44] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [15/Jul/2020 16:20:44] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [15/Jul/2020 16:20:44] "POST /_dash-update-component HTTP/1.1" 204 -
127.0.0.1 - - [15/Jul/2020 16:20:44] "POST /_dash-update-component HTTP/1.1" 200 -


None <class 'NoneType'>
'DataFrame' object has no attribute 'x'
None None
None <class 'NoneType'>


127.0.0.1 - - [15/Jul/2020 16:20:58] "POST /_dash-update-component HTTP/1.1" 200 -


{'subType': '>=', 'type': 'relational-operator', 'value': '>=', 'left': {'subType': 'field', 'type': 'expression', 'value': 'x'}, 'right': {'subType': 'value', 'type': 'expression', 'value': 10}} <class 'dict'>


127.0.0.1 - - [15/Jul/2020 16:22:18] "POST /_dash-update-component HTTP/1.1" 200 -


{'subType': '>=', 'type': 'relational-operator', 'value': '>=', 'left': {'subType': 'field', 'type': 'expression', 'value': 'x'}, 'right': {'subType': 'value', 'type': 'expression', 'value': '10 and <=22'}} <class 'dict'>


In [83]:
import io
import pathlib
h = pathlib.Path.home()
nym = open(f'{h}/downloads/nymex.settle.20200619.s.csv','r').readlines()
sio = io.StringIO()
sio.writelines(nym)
sio.seek(0)
df = pd.read_csv(sio)


In [84]:
len(df),len(nym)

(38157, 38158)

In [124]:
import zipfile
z= f'{h}/downloads/nymex.settle.20200619.s.csv.zip'
f = zipfile.ZipFile(z).open('nymex.settle.20200619.s.csv')
nym2 = [l.decode("utf-8")  for l in f]
sio2 = io.StringIO()
sio2.writelines(nym2)
sio2.seek(0)
df2 = pd.read_csv(sio2)


In [125]:
df2

,BizDt,Sym,ID,StrkPx,SecTyp,MMY,MatDt,PutCall,Exch,Desc,...,HighBid,LowBid,PrevDayVol,PrevDayOI,FixingPrice,UndlyExch,UndlyID,UndlySecTyp,UndlyMMY,BankBusDay
0,2020-06-19,QH,QH,NaN,FUT,202007,2020-06-29,NaN,NYMEX,NaN,...,NaN,NaN,0,1,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-06-19,QH,QH,NaN,FUT,202008,2020-07-30,NaN,NYMEX,NaN,...,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-06-19,QH,QH,NaN,FUT,202009,2020-08-28,NaN,NYMEX,NaN,...,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-06-19,QH,QH,NaN,FUT,202010,2020-09-29,NaN,NYMEX,NaN,...,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-06-19,QH,QH,NaN,FUT,202011,2020-10-29,NaN,NYMEX,NaN,...,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38152,2020-06-19,MDB,MDB,NaN,FUT,202007,2020-08-04,NaN,NYMEX,NaN,...,NaN,NaN,0,10,NaN,NaN,NaN,NaN,NaN,NaN
38153,2020-06-19,MDB,MDB,NaN,FUT,202008,2020-09-01,NaN,NYMEX,NaN,...,NaN,NaN,0,10,NaN,NaN,NaN,NaN,NaN,NaN
38154,2020-06-19,MDB,MDB,NaN,FUT,202009,2020-10-02,NaN,NYMEX,NaN,...,NaN,NaN,0,10,NaN,NaN,NaN,NaN,NaN,NaN
38155,2020-06-19,MDB,MDB,NaN,FUT,202010,2020-11-03,NaN,NYMEX,NaN,...,NaN,NaN,0,10,NaN,NaN,NaN,NaN,NaN,NaN


In [165]:
import zipfile
# z= f'{h}/downloads/nymex.settle.20200619.s.csv.zip'
z= f'{h}/downloads/df_avg_implied.csv.zip'
zo = open(z,'rb').read()
zoio = io.BytesIO()
zoio.write(zo)
# f = zipfile.ZipFile(zoio).open('nymex.settle.20200619.s.csv')
f = zipfile.ZipFile(zoio).open('df_avg_implied.csv')
nym2 = [l.decode("utf-8")  for l in f]
sio2 = io.StringIO()
sio2.writelines(nym2)
sio2.seek(0)
df2 = pd.read_csv(sio2)


In [173]:
zo

b'PK\x03\x04\x14\x00\x08\x00\x08\x00\xe9x\xf6P\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x12\x00\x10\x00df_avg_implied.csvUX\x0c\x00f\x8e\x18_f\x8e\x18_\xf5\x01\x14\x00m\x9aI\xaf\xe3\xba\x92\x84\xf7\xfc-\x17\x0f")Q\xd2R\x9e\x8e]\x1e\xcb\xd6\xf1\xf1\xa9M\xa3W\xbd\xe9\x87\xb7h\xe0\x01\xfd\xef\xfb\x8bL\xca\x05\xf4\xad\xc2\x1dPa\x9b"s\x88\x8cL\xea\x7f\xfe\xf7\x9f\x7f\xfd\xe7\xbf\xff\xeb?\xfe\xfd\xaf\xff\x0e\xd3t\xfa+\xfe\xa3i\x87\xdc\xb6\xa5IM\xd7\xf4Cl\xc3e}\xda\xff\xd5\xfcc,m\xee\xba!uq\xecrL]\xb8\xafO\x82sj\x9a\xbe\x8b\xddP\xc6\xd86}\xd8m\x85\xc6qL]\xeec3\xe6\xbe\x0b\xd3m\x12\xd8\xb4i\xec\x07~\xc0\xef\xdbq\x0ck[a\x18Y\xb7\xeb\x97ucX_\xe7o\xe1}\xd74z\xd0\xe7d_+c*m\xd3\xa42t][\xfap|<\x047c3\xe4\xb1\xd5\xda\x85\x9f\x84\x8f\x9b\xe0~\x04\xebs\x9f\x9b\xd6\xd0\xd3J\xe0\xc0wb\xec\xda~`\xc3%\x87\xf9v\x17\\\xfafL\xb1O\x86\xf2\xc0\xf3\xfdjp[0\xc1\x18\x1bG7\xaf\xb5\xd0NF\xe1\xebZ\xf5\xfcq6\x88\xadw\xa5\xe9\xfcQ\x9f:k\xdf\xe6XJ\xc9\xcd\xd0\xc9\x02\x11\xd4\xe0T\x9a\xbe\x8c\xdd\xd0\xc8\xb6\xa9\x

In [167]:
zoio.read()

b"PK\x07\x08\xf0\x0c\x0e\xef0\x16\x00\x00\xa8-\x00\x00PK\x03\x04\n\x00\x00\x00\x00\x00r|\xfbP\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\t\x00\x10\x00__MACOSX/UX\x0c\x00\x87,\x1f_\x87,\x1f_\xf5\x01\x14\x00PK\x03\x04\x14\x00\x08\x00\x08\x00\xe9x\xf6P\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x1d\x00\x10\x00__MACOSX/._df_avg_implied.csvUX\x0c\x00f\x8e\x18_f\x8e\x18_\xf5\x01\x14\x00c`\x15cg`b`\xf0MLV\xf0\x0fV\x88P\x80\x02\x90\x18\x03'\x10\x1b\x01q\x17\x10\x83\xf8{\x18\x88\x02\x8e!!AP&H\xc7\x0c VAS\xc2\x88\x10\x17M\xce\xcf\xd5K,(\xc8I\xd5+,M,J\xcc+\xc9\xccKe(\xd470\xb00\xb26M3\xb4\xb0H57\xb7\xf6\xcdL.\xca/\xceO+\x89\xa902p\xadHN\xcd\xb1f\x00\x00PK\x07\x08]5\x10#o\x00\x00\x00\xbc\x00\x00\x00PK\x01\x02\x15\x03\x14\x00\x08\x00\x08\x00\xe9x\xf6P\xf0\x0c\x0e\xef0\x16\x00\x00\xa8-\x00\x00\x12\x00\x0c\x00\x00\x00\x00\x00\x00\x00\x00@\xa4\x81\x00\x00\x00\x00df_avg_implied.csvUX\x08\x00f\x8e\x18_f\x8e\x18_PK\x01\x02\x15\x03\n\x00\x00\x00\x00\x00r|\xfbP\x00\x00\x00\x00\x00\x00\x00\x00\x00

In [168]:
df2

,sym,avg_vol
0,AAL,1.048345
1,NCLH,0.964356
2,RCL,0.932008
3,FE,0.919925
4,APA,0.904298
...,...,...
496,COST,0.191384
497,PEP,0.190468
498,JNJ,0.189034
499,PG,0.185250


In [140]:
import re
re.findall("\.zip$",'df_avg_implied.csv.zip'.lower())

['.zip']

In [177]:
import base64
c = "UEsDBBQACAAIAOl49lAAAAAAAAAAAAAAAAASABAAZGZfYXZnX2ltcGxpZWQuY3N2VVgMAGaOGF9mjhhf9QEUAG2aSa/jupKE9/wtFw8iKVHSUp6OXR7L1vHxqU2jV73ph7do4AH97/uLTMoF9K3CHVBhmyJziIxM6n/+959//ee//+s//v2v/w7TdPor/qNph9y2pUlN1/RDbMNlfdr/1fxjLG3uuiF1cexyTF24r0+Cc2qavovdUMbYNn3YbYXGcUxd7mMz5r4L020S2LRp7Ad+wO/bcQxrW2EYWbfrl3VjWF/nb+F91zR60OdkXytjKm3TpDJ0XVv6cHw8BDdjM+Sx1dqFn4SPm+B+BOtzn5vW0NNK4MB3YuzafmDDJYf5dhdc+mZMsU+G8sDz/WpwWzDBGBtHN6+10E5G4eta9fxxNoitd6Xp/FGfOmvf5lhKyc3QyQIR1OBUmr6M3dDItqmE3XwwuOmbboy9H2HzvAjkqHnk1HlIg+zw9X0RXsae4zY55XYoOY1hOh0FD33fsn6OQ+aZY9iY2cqQY8P39NSsJ96e8mbBDy3rZMzBQl1YaXulsIHYsOGxLzLQbrp8GI4NCIChzXJKCpvDw1bpGmwxYlYzUhvOe9tKm7ohltGt1MRwuT4Nbkpye4TnYZI1MVBm1yPHGfmwhO3rpvgpBEpp+eOWSuFxs30QAbj/t0+2886WiZnQi0NJ/cgvw+nbnhdj27XN0HAkfB7D42SLNF1HbLC8xcwY9l9nh2W+Gty76wvM9qpdc2j+xDCdN4KHthuHYWhKNd5+dTBYoRW77BFewvn6MLjp+qY0TWdnL2H+mhV2HINvkxBubXZ9WRs8jEPG+ZkMIkjC3vzY9ZG/sBPzDIsTjtp2R4CM5OuQeGQTcdnJHorfS7aU1O5jOH0+De5jmzlKOyiocjg9/dtkYIlNYqkxpbA3qxIAKRMoBLeiNWO/lcFNxtgZm3uQvO5mKxZg+SGlVMw3H1tDu4LH6tnJg6efnVwZa8KbYdfnyWD2gGUbZ46B0P5QyHetUiY34+h2DJevx+Q4th5jTbMS7maqllSIMFhN6bAzEEIYhzchnKe7o6QS2405c1CM/ZgNJoqqG0VC4Xq0w2Ssxicxml1z+NqsHe4baIidjKUZ27DZmVHTiOPbNg/VTvPL1k49iWuQr727VzgWGDONqR2atg/r6W47TCWmHsIiXjFXGz5Psx0deiUheXYiwpsxnG+2l5T5ZsY1sfQK7t365XDbVseYvW/Tt3+dtZuB1YkdzAUfXXw3HA+SLV0kAlI43D5uDpMlAxUi9UTXEE7Hn4KhukZc5QQRHt9mcaXo6MjudLg7lEnfRkGBAduwO8wWUhGqzemd1tPK80xpzfHSkNqekAuH5y+DE/aOVgbkysfh6YuQIW0pfe4oCANk9H2qMGGcnUOPB0ua2MA6TWVhonK/mizOmhFzmIe06/3W/NgMLVZi3axlc7icHv7lIRM6sF/r1WB/t21gCZwz8hOtUsJldfoTvL7u/g7D/XejEswMB/TwX0vJCp973wkWieP76GHefBjMXygJMdvuWcKCFUqNMdUKvH5dHVOUtxEC6wnu8PREJ1XaqERqFGUpPC/+7WaE0hZzREWBw0o7BQCU27dhf9CWVQcoBFn1mMRJrYrE+uifNGloxa7m3fXmy9CeP7CcuQKCDMftt+GlI59KVwt6sXUm+0QchfcrSedw3xmcolOlIufw4ZAKKlvxJKZ2bk8n+0D2GC0wLazC6qjAhA47nNl0C/HcXpu/w30fLnbUAbru86hzyTRhe7WnQkJIiY6ylSnBbThtHeb7xA2JLHt1pPDl/Ed8us1P+yD2ZAhGcRsTQs/N5B/wOBllNLcQAJPSkgLat6QW0sKW6slAM2XPqm0afPMYPBxmg1kCcgJuZTwkiwUduPynCLH44LRfZrSehMLiNdJh5fOvi+PUP1FzSxkbdeC7sQ2e7mBbkauH2eVqHidXCAN+NI4EZlSRnRxPaEsKqgUUfr3UcxV9QqTFJNoeg8kt0IFK1fTGzmG6bGdHOWCDfZoojRb2HjOlQNYDpak3+iPvDheLGpREYSP9wrfTeW2RUGDz1Hl9ZPmf9+fVYYphykgPJVbThut57XhU5rWewENY78znJSFIVN0t5ln+6350PJLwEgoW3Tz2YC4v0LVCzfZOqd2bDCL9qd0QDT5RHW/D/WoFCruqoo6LrAiHy/rb8Ux24xWTXymcX0Z4badnYn6XgZnCunI4oVLf5SXsT746jCAXlqqJISs7VddCnCKLyuoroyV+SNJSLBr9hkS+nK5Hx4k6KsOodFD2rOeT4/FdhmffB0uqoDUiZra9PvvzIhTURVyB6BxTuO8tTtld46JOa8B4X3tzHQEgWmgXHX3aXhzmFPzc6wKwBWOr3IMOKIFYn+bHIo5HwjxowWiHQS4/LKI5L1JZCRtVuSCxq3+fPZNJ5ECtdPcfFl1knToMMSU/CLf92VESshslP5NJgsdlchynoRYQcsoxStrX0XFFCRlPaCSY9u6EgnORsKT8wu6/VndfR8FJ/MKaKIAeZTZZVCgZf0uf/e2ngziSyvNue778u4M1EUs3ksPNIxoKZJkIZbcmcr6sdHB+6Zx+EX2PrcUt/o5aB4ksc4Xb9+3kuH5MwNEANrDvep4sglBECBCsNSxxsd6bfSm55S3PJLkwgTmVAkPThoSPvefv83R1nG9SVqLJJUSXBbSc8G7GwpfnEH/HnZF0Y1vtSIquHMYDEGnpa01ZHS9ugcZoNMEEqstduGzNq2oiqlYMazs+1Y5ggb5pQRAe9CMrhzO1eum52mCaiJggdGpwWjFZudPEexBdTc0UVl+Tw6gW6+pcA9xWaztNIgHtHC5GnpMvzkZZZBEvbfh1uJr9YNwkLlps8phfDsP/eMYrG3S2uvuJ8kBgVYKm4ptBpIXRHbXu0Ei5MxESmPAdVvP8ZZ6J2jP9L64hjtB+Xuw4I+Txu4MPz7tvJfY6PJFhao0qSFLY+qhNmlza1MaUCeVid/JfgEcFClWzSDmsXhZEhDnMnxHjbpuN00hMkuGlaiF2ert/Ok7QYvUhGnGFjVcvipD6avSo+YlOy5waY6FhQT1JqUmHf1YUmUIiWl9LDDkV0YqTilV9xAQNvyquvEJ4uIaWYDan0lFgn1iJlYZjsmdqldi+ZyYUHUfd8k5+s4nU/weu56cFkR6H16yvL1JS0+GX40idwv6ogpB/DsezWZYikwZKB/2dxARS5Tz5BokdBGxUGpHpfGCaGRyj1tw0Fnl4IuLS7GTuG1odrg7DWFBW21vian3zHFKzownpanMRdjsrlya9NKZaJOVq9e0wpTvJyhaqtACP9cE/IJdI4FJT/Xw6O9xpBZYxLu3D8WRjCT7Ians1OVEKhfWlwop4/uvc0IbL59ZxJCIio7EcJsueZ/8+8hiOGTwvE2XAUCKUJGmXPjLMu5fhRZVbDKPlaRFOhuJmnrnMlwoxMDuOUqcdreMDHL5xG4jX2DQrY+kh+IQEOKkUFVMvRNJ1PjwMJ6Jx7FIGwvzD90Ib2SzxhcQjNz6u/kGUsqM3NMETDqfzxXB4iZBfeAmRu7kbzplqX1YFz/x0l6CBxI9j7Y/D9uEbMt1OkenURmDj1/fpj/jnQ7mglOlMmCiSpXEnS3rICoWOjBlorFux9e1yd7yBHdpUBxVE4MHX6SllNFwu3zHE9uQwPEPDyhNMUE2bx9HwQgLqqBpJtUPYPu8fhncieMkwDec47/5xWPsHLNDgcXcB+zydLHg4vqkeRLy7gD7ILEEw0brCzmrN4xA+bmZpMYcTMY0VgR0Om9fLP2CfveYZ1iKhCizc4NfBSoMX1zA9vx3mr3h3gJF5LIXky+zDNpDdy+Qthtfp8nI8UROxTHFluZ5fD8cbTT8XNkIvHM2/FuQcKXmXET5Mdqk4j+pMlw50Ok6WimhQcsV8IL9I18/+AXTEQxUsKlZt2Po6iHeioHJ0G2aPEvQV4Zlqb9rxVEPJrYF1XJCX8MNBxDbKolhzQcRuZvMHmpcSK47NeB3R6sdJmr1RcXzIE7xIIYoII/LeBT06fLczmBiTQ51XxvBzffXDNO0gteA2R1c7iJ5YIowyettOFmJ8D33aR880TYT2DuPd9xwcPbvZ/BH+OltcI3F6Y4LOekCY1U5Je8naqW1dF1JxnLKwEkG31C0IfWMVA7+MGCz3te8iDw5/gh8rJ1DNHrET5cGkWDj51rOUKzVhlPPhQ6fVorHSMC5zmvDDTU7nxva6xQCaFj2e/oHqgYjWOlLY+eawGv6eVtWUR7if3TCNTSZIIQUQzr/Y1J+o4YBLs45pwrSzYKERJbKWEk0AzOfPh39AolOQ62A7bH/6OpA2xtdkv5X7jo9Ph9EDFM5sZQoeuJpIyZpea3ywNFGbjwpr0KZYt+3n8OO4tzxFuRQ6o2H0BOvCdVP3yXfZYF8jOIetMzHErRBo6iOUSc6skIMmHzCx0THU574CVxZl0y82LJptui+xjl+RMsUDc/9pQawpJdRB0e91z4NWt8rQJZtLDGrO2GgJ56PnCOaRemiHbFcqND033w/uTu9ZZU/Xf3ec3XRvotRCj4PjRAhJOCQTLuGw9eVpx7JKyVAkq8Pj++iwmoqxjiAifGtTwqzxlUblHuTh4+urolHalEpirRwN9WxubHW7odlW1IytC8f92uE+a34/GqmOqC03AkyHGfsqrCGryVH8tLTHyJXLwxehNrG1oaq+sLHM1GAyicQ0FaWy7DZ2TE1ISQ/sNYhboVmrN+rs+9+dPYnm3+aRRcMaz0xqhz+RfOriMnQaw2ljVxQK0VzbEovJk03pdQ8YoW9KnV9F3H2DmjWJJPEoEidMLmtoBUdNpUYvxmH98J0QocMoprVhCPF7P337B0VXIDCllTS8vJsdJzOxU6yXOYebo+QdujxbR0l+nLf12wgpWuo6lICv7K4p0wzQ2A12D2iN4PNuglgXRVK39YKFqno/L7gO1NV7T9jW12+SOGsYvAEN96slnwYJzuG6yRyzhpSOk1mQxjJVejnvZ7WqDWey31Bsty75TL0ogokhrBM+fBE8qsrfGx3y0Gm1dhwVYfMKa0TJpPX+yz+ILYdt31Pk9fXqOEXeYtXa4XBxjaJa8Z7KSDLdbV4D3ipfNKRX9Ibt4VXhXAdKVlOnzWrrOFxDVySBIY6/20hEMMd8X+pBD3VxojH1Q61E4TxvKkwBovnvk8nhsN5c3AhdI+PU+TrduaWjSKrvlnlYWHmM8TQ11GNfq9B5evwJ/rheXQ9gJA7TqDaOajD4geUpDy2k9cKFJMLtl+FKF0l+Ky4p/Li5V23+gDVLsthY3w1Og91VaaSrGhJO1j8iBumq6eabaFOR8HWaK45Ddbvn/Yzt0nHikPwlyWhq6MQNRAi1JvR8sLK1K3Tgxv5x0tTdx8VmHJkgH37vEMs8HNVA++1Tujwj2CSSqi2eRdHt7oskm4otFyNhO1vGJOn6uNxIdtRDjxdwdIhCQHmZggvpREnthlpj+F+YnHr+hh+qFQldTJ79doVgn6abL0Q3SyJBWEiIRI96uvguGzGM7spsGBPO17+jMOG0sSjF8VKry3Afwn94ckBWGmhI1A5RKbw93x1HWrLVxqZprPP0hojGF6qGO7xF64OjvarVMsGHaJ3ziPVOs2+8oisLMnj1aTbDE81bivYtseQu0XsVaOMOZ9Hxs835Y3YcdaQrOGvwqf0v32SnUF/uk0imp6+OKB6S7oRd6XwcTpZ8UWM+Yn5w6ifGrpbZCNpO8t01DbG3vV8cp3RKHDupoiVtMxrEsQBF3MQULvxpHNxomC8RaSHe6NLVYKpm8248kCHGD8qJvuuWEd3Kv0pJJ4uwevS59erucJKLiw8jkWJrV4watw7d+z5YLZbDVO9OQt2aUCSCK33OgRtIVrvh7lXzjWcJXYpPquM7dvPtwkcz2HrrLdYnZjyDKeyqvwSy5q+DvcWw8Q8gn9TUcW8Kl6uvnzLCpJDuUe/UhFXdpqbakI930IVT2eV8RqhnsWy9hKLe+uKSMJrikMbUW43wfjlOGYPeawenwucGVtvVqYeDJRDc4WbXeeBw2/B71j/fja+SmktYvF7opfBpKjtJCrkWspnZr/nhaAcvv+/mws3u55NGJRrIdnphZSSo1xXHP5Lli1Sf70+De3k6665f3XnY+trQAovQsdjkLfxY7ec/4buDf70Mw/KKkfXHp8Ol4nZDr7mUhgXBd4K44d+yiPrH3g/fqYhJ65ssQsyYvku6StBdaL1/4kRbhzXWjrn6qISN3UgQEFklb+ExWGO18pM2klUl1Zs5SS77AaE+DBqpRt2806o+bh+HP36wOn87rvbrnTktfcbF8aRXCnJjLoElJxNv1KbS6q2OzmiPA9sVMnC2qXwzmhiHIjYOq8zmOlDVzPO+NpwyU+Amn8ChZ6ZfZh66tU7dR70oCPPLNwOCxlGeSLp+mv5PigoIopTlGmP78sU7fcTzfEqo+9K94xqmK/p0QUV67O/mKytCwPWOTyMXN40mIuUti8J82BnMouX9EhihunUL6Ipa9bkOFI7uD83ax2hjfBsRWw1CmTYd2VFHV4SH9XaqMo00AphELZ2jWUVEsrxioRibNmeH8VdbX++CocNq46t0HK/t6uU/iTr9sB3qLq2VFnJBE272qh1wp6aA/0XT9OfN7LBWjjjIWA9JcPan6uKFUMFg4l+sZZ5WYerVdqHVjavXC06HXAu8r6sbYqJtaRf5ojULIkxq8vucSOWHb1v6o6vvFojDT2dfSFMACsxgerxwHitKSYMDGoNSrz278HQUHUOgRH9tSy+wvBzX2KCtl2FtuF0sJoq9naWbWRsEhctm+um4pB/OLOZRaRnbPMGvwcly2xIeni1F16nZLSXSD7cP/3qnm3VdINgtWZi2/vVWg39W8eugcKmw5tV6t86cMYTLscJiHhq7tg6apg+HdT0K/djNLjrRzVU0k6Cu51In7WvTchpLURsRso1u7EjRyQlAV97q4NvWbiGIrskN1ugqB7K32xOWuUlY6p4o/74nyixztos2VIrG3ZKMlZSn1ex4ryv7vr62g6C3G25UhgayhF523T7N1woTvJLcpb4zZ+9qSZRo3FpfVAu3td1bJSU4MMFmoyJK886XgZOX10r1GlswJQ7VkQAkY/KmQyXVN5PUHtolpV1nGajSrUGET4050FeFSThdw/qbrKgYu8yjmcvJ3m9YrjKu/uYrWYT6GJr33G43meaWFhr0o3q9Qqu6tsAmGjUCzvVVrhQeP2zz2cZciSDws543p1+GD4MW9gEznjofjR7xnd6AGuutq95tcVy1He2QfdbeoerNs1nzNipkA71bPr1MkMsgSRWo1kM8vr44HvVmFCpNs5OwcQxDsUvjBr0fs3VYIdTryTb70sxRMLHq9dC9SjaZ73DqWKPX3g/0co2D9Jpc5+87SoWtbBECOqsxQlhotkqg2q6j3nRpuuWGEnLcmiyJ6pL69/sZtHQ2zwTXm8yli8tY9Gh7wU58dViGXm3YfFoQ6L0ComV5WxPykStgRaIjv9/VgfD9PQxOqKYcLWs1aaCY3AxWqEiJ9T4VDD8umjkrupRf1GAzZDAFZu9r8ARiyV9dub5Uv3S+9wXX/wFQSwcI8AwO7zAWAACoLQAAUEsDBAoAAAAAAHJ8+1AAAAAAAAAAAAAAAAAJABAAX19NQUNPU1gvVVgMAIcsH1+HLB9f9QEUAFBLAwQUAAgACADpePZQAAAAAAAAAAAAAAAAHQAQAF9fTUFDT1NYLy5fZGZfYXZnX2ltcGxpZWQuY3N2VVgMAGaOGF9mjhhf9QEUAGNgFWNnYGJg8E1MVvAPVohQgAKQGAMnEBsBcRcQg/h7GIgCjiEhQVAmSMcMIFZBU8KIEBdNzs/VSywoyEnVKyxNLErMK8nMS2Uo1DcwsDCyNk0ztLBINTe39s1MLsovzk8riakwMnCtSE7NsWYAAFBLBwhdNRAjbwAAALwAAABQSwECFQMUAAgACADpePZQ8AwO7zAWAACoLQAAEgAMAAAAAAAAAABApIEAAAAAZGZfYXZnX2ltcGxpZWQuY3N2VVgIAGaOGF9mjhhfUEsBAhUDCgAAAAAAcnz7UAAAAAAAAAAAAAAAAAkADAAAAAAAAAAAQP1BgBYAAF9fTUFDT1NYL1VYCACHLB9fhywfX1BLAQIVAxQACAAIAOl49lBdNRAjbwAAALwAAAAdAAwAAAAAAAAAAECkgbcWAABfX01BQ09TWC8uX2RmX2F2Z19pbXBsaWVkLmNzdlVYCABmjhhfZo4YX1BLBQYAAAAAAwADAOYAAACBFwAAAAA="
content_decoded = base64.b64decode(c)
# Use BytesIO to handle the decoded content
zoio2 = io.BytesIO(content_decoded)
f = zipfile.ZipFile(zoio2).open('df_avg_implied.csv')
nym2 = [l.decode("utf-8")  for l in f]
sio2 = io.StringIO()
sio2.writelines(nym2)
sio2.seek(0)
df = pd.read_csv(sio2)



In [178]:
df

,sym,avg_vol
0,AAL,1.048345
1,NCLH,0.964356
2,RCL,0.932008
3,FE,0.919925
4,APA,0.904298
...,...,...
496,COST,0.191384
497,PEP,0.190468
498,JNJ,0.189034
499,PG,0.185250


In [183]:
app = dash.Dash()
inp = dcc.Input(id='myinput',)
app.layout = html.Div([inp,html.Div(id='c1'),html.Div(id='c2')])

@app.callback(
    [Output('c1', 'children'),
     Output('c2', 'children')],
    [Input('myinput', 'value')])
def update_graph(v):
    return v,v.upper()
app.run_server(host='127.0.0.1',port=8844)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8844/ (Press CTRL+C to quit)
127.0.0.1 - - [28/Jul/2020 15:11:16] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/Jul/2020 15:11:16] "GET /_dash-component-suites/dash_renderer/polyfill@7.v1_3_0m1588634869.8.7.min.js HTTP/1.1" 200 -
127.0.0.1 - - [28/Jul/2020 15:11:16] "GET /_dash-component-suites/dash_renderer/react@16.v1_3_0m1588634869.13.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [28/Jul/2020 15:11:16] "GET /_dash-component-suites/dash_renderer/prop-types@15.v1_3_0m1588634869.7.2.min.js HTTP/1.1" 200 -
127.0.0.1 - - [28/Jul/2020 15:11:16] "GET /_dash-component-suites/dash_renderer/react-dom@16.v1_3_0m1588634869.13.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [28/Jul/2020 15:11:16] "GET /_dash-component-suites/dash_table/bundle.v4_6_2m1588634869.js HTTP/1.1" 200 -
127.0.0.1 - - [28/Jul/2020 15:11:16] "GET /_dash-component-suites/dash_html_components/dash_html_components.v1_0_3m1588634869.min.js HTTP/1.1" 200 -
127.0.0.1 - - [28/Jul/2020 15:11:16] "GET /_dash-component-su

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/flask/app.py", line 1936, in dispatch_request
    return self.view

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/flask/app.py", line 1936, in dispatch_request
    return self.view

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/flask/app.py", line 1936, in dispatch_request
    return self.view

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/flask/app.py", line 1936, in dispatch_request
    return self.view

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/flask/app.py", line 1936, in dispatch_request
    return self.view

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/flask/app.py", line 1936, in dispatch_request
    return self.view

127.0.0.1 - - [28/Jul/2020 15:11:16] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [28/Jul/2020 15:11:19] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [28/Jul/2020 15:11:19] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [28/Jul/2020 15:11:19] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [28/Jul/2020 15:11:19] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [28/Jul/2020 15:11:20] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [28/Jul/2020 15:11:20] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [28/Jul/2020 15:11:20] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [28/Jul/2020 15:11:23] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [28/Jul/2020 15:11:34] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [28/Jul/2020 15:11:34] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [28/Jul/2020 15:11:35] "POST /_dash-update-component HTTP/1.1" 200 -
127.

In [201]:
from dashapp import dashapp2 as dashapp
import importlib
importlib.reload(dashapp)
inp = dcc.Input(id='myinput')
dap = dashapp.DashApp()
layout = html.Div([inp,html.Div(id='c1'),html.Div(id='c2')])
def _dd(input_data):
    return [input_data[0][::-1],input_data[0].upper()]
d = dashapp.DashLink([(inp,'value')],[('c1','children'),('c2','children')],_dd)
dap.add_links([d])
dap.create_app(layout,app_port=8844)


2020-07-28 15:43:33,303 - root - INFO - This app will run at the URL: http://127.0.0.1:8844


 * Serving Flask app "dashapp.dashapp2" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8844/ (Press CTRL+C to quit)
2020-07-28 15:43:33,312 - werkzeug - INFO -  * Running on http://127.0.0.1:8844/ (Press CTRL+C to quit)
127.0.0.1 - - [28/Jul/2020 15:43:36] "GET / HTTP/1.1" 200 -
2020-07-28 15:43:36,207 - werkzeug - INFO - 127.0.0.1 - - [28/Jul/2020 15:43:36] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/Jul/2020 15:43:36] "GET /_dash-layout HTTP/1.1" 200 -
2020-07-28 15:43:36,385 - werkzeug - INFO - 127.0.0.1 - - [28/Jul/2020 15:43:36] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [28/Jul/2020 15:43:36] "GET /_dash-dependencies HTTP/1.1" 200 -
2020-07-28 15:43:36,386 - werkzeug - INFO - 127.0.0.1 - - [28/Jul/2020 15:43:36] "GET /_dash-dependencies HTTP/1.1" 200 -
2020-07-28 15:43:36,419 - root - DEBUG - ****************************** execute_callback no data for [('c1', 'children'), ('c2', 'children')] ***************************************
127.0.0.1 - - [28/Jul/2020 15:43:36] "POST /_dash-update-component HTTP/1.1" 204 -
2020-07-28 15:43:36

In [219]:
?dashapp.make_dashtable

In [ ]:
from dashapp import dashapp2 as dashapp
import importlib
importlib.reload(dashapp)
inp = dcc.Input(id='myinput')
s = dcc.Store(id='mystore')
d1 = dashapp.DashLink(
    [(inp,'value')],
    [(s,'data')],
    lambda i:[pd.DataFrame({'d':[i[0],i[0][::-1]]}).to_dict('records')])

t,tlink = dashapp.make_dashtable('dtable',df_in=pd.DataFrame(),
                input_store=s,update_columns=True)


dap = dashapp.DashApp()
layout = html.Div([inp,html.Div(id='c1'),html.Div(id='c2'),t,s])

d2 = dashapp.DashLink([(s,'data')],[('c1','children')],
                     lambda i:[i[0]['d']])
d3 = dashapp.DashLink([(s,'data')],[('c2','children')],
                     lambda i:[i[-1]['d'][::-1]],
                     )
dap.add_links([d1,d2,d3,tlink])
dap.create_app(layout,app_port=8844)


2020-07-28 19:38:47,124 - root - DEBUG - dtable entering create_dt_div
2020-07-28 19:38:47,126 - root - DEBUG - dtable exiting create_dt_div
2020-07-28 19:38:47,140 - root - INFO - This app will run at the URL: http://127.0.0.1:8844


 * Serving Flask app "dashapp.dashapp2" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8844/ (Press CTRL+C to quit)
2020-07-28 19:38:47,148 - werkzeug - INFO -  * Running on http://127.0.0.1:8844/ (Press CTRL+C to quit)
127.0.0.1 - - [28/Jul/2020 19:38:53] "GET / HTTP/1.1" 200 -
2020-07-28 19:38:53,268 - werkzeug - INFO - 127.0.0.1 - - [28/Jul/2020 19:38:53] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/Jul/2020 19:38:53] "GET /_dash-layout HTTP/1.1" 200 -
2020-07-28 19:38:53,368 - werkzeug - INFO - 127.0.0.1 - - [28/Jul/2020 19:38:53] "GET /_dash-layout HTTP/1.1" 200 -127.0.0.1 - - [28/Jul/2020 19:38:53] "GET /_dash-dependencies HTTP/1.1" 200 -

2020-07-28 19:38:53,372 - werkzeug - INFO - 127.0.0.1 - - [28/Jul/2020 19:38:53] "GET /_dash-dependencies HTTP/1.1" 200 -
2020-07-28 19:38:53,406 - root - DEBUG - ****************************** execute_callback no data for [('mystore', 'data')] ***************************************
127.0.0.1 - - [28/Jul/2020 19:38:53] "POST /_dash-update-component HTTP/1.1" 204 -
2020-07-28 19:38:53,408 - werkzeug - I

_dash_table_update_paging_closure: page_current:0 page_size: 20


127.0.0.1 - - [28/Jul/2020 19:38:55] "POST /_dash-update-component HTTP/1.1" 200 -
2020-07-28 19:38:55,179 - werkzeug - INFO - 127.0.0.1 - - [28/Jul/2020 19:38:55] "POST /_dash-update-component HTTP/1.1" 200 -


...

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/flask/app.py", line 1936, in dispatch_request
    return self.view


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/flask/app.py", line 1936, in dispatch_request
    return self.vie

2020-07-28 19:58:21,799 - dashapp.dashapp2 - ERROR - Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/flask/app.py"

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/flask/app.py", line 1936, in dispatch_request
    return self.view

127.0.0.1 - - [28/Jul/2020 19:58:21] "POST /_dash-update-component HTTP/1.1" 500 -
2020-07-28 19:58:21,801 - dashapp.dashapp2 - ERROR - Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File